In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
fashion_mnist = keras.datasets.fashion_mnist

In [3]:
(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [4]:
# scaling down the images in the range of 0-1 so that it help us to quickly train our model
train_images = train_images/255.0
test_images = test_images/255.0

In [5]:
train_images[1].shape

(28, 28)

In [6]:
# reshaping the images
train_images=train_images.reshape(len(train_images),28,28,1)
test_images = test_images.reshape(len(test_images),28,28,1)

In [7]:
def build_model(hp):
    model = keras.Sequential([
        keras.layers.Conv2D(
            filters = hp.Int('conv_1_filter',min_value = 32, max_value = 128, step = 16),
            kernel_size= hp.Choice('conv_1_kernel',values = [3,5]),
            activation='relu',
            input_shape=(28,28,1)
        ),
        keras.layers.Conv2D(
            filters = hp.Int('conv_2_filter',min_value = 32, max_value = 64, step = 16),
            kernel_size=hp.Choice('conv_2_kernel',values = [3,5]),
            activation='relu'
        ),
        keras.layers.Flatten(),
        keras.layers.Dense(
            units=hp.Int('dense_1_units',min_value = 32, max_value = 128, step = 16),
            activation='relu'
        ),
        keras.layers.Dense(10,activation='softmax')
    ])
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values = [1e-2,1e-3])),loss='sparse_categorical_crossentropy',metrics=['accuracy'])
    return model


In [8]:
from keras_tuner.tuners import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [9]:
tuner_search = RandomSearch(build_model,objective='val_accuracy',max_trials=5,directory = 'output_1',project_name = "Mnist Fashion")

INFO:tensorflow:Reloading Oracle from existing project output_1\Mnist Fashion\oracle.json
INFO:tensorflow:Reloading Tuner from output_1\Mnist Fashion\tuner0.json


In [10]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split =0.1)

INFO:tensorflow:Oracle triggered exit


In [11]:
model = tuner_search.get_best_models(num_models=1)[0]

In [12]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        48432     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,147,370
Trainable params: 3,147,370
Non-trainable params: 0
_________________________________________________________________


In [13]:
# retrain the model according to the results we get after random search
model.fit(train_images,train_labels,epochs=10,validation_split = 0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 233s 137ms/step - loss: 0.1234 - accuracy: 0.9534 - val_loss: 0.2698 - val_accuracy: 0.9120
Epoch 5/10
1688/1688 [==============================] - 206s 122ms/step - loss: 0.0840 - accuracy: 0.9680 - val_loss: 0.3079 - val_accuracy: 0.9137
Epoch 6/10
1688/1688 [==============================] - 221s 131ms/step - loss: 0.0577 - accuracy: 0.9786 - val_loss: 0.3568 - val_accuracy: 0.9117
Epoch 7/10
1688/1688 [==============================] - 244s 144ms/step - loss: 0.0402 - accuracy: 0.9853 - val_loss: 0.3891 - val_accuracy: 0.9098
Epoch 8/10
1688/1688 [==============================] - 239s 142ms/step - loss: 0.0315 - accuracy: 0.9888 - val_loss: 0.4541 - val_accuracy: 0.9115
Epoch 9/10
1688/1688 [==============================] - 222s 131ms/step - loss: 0.0228 - accuracy: 0.9921 - val_loss: 0.4893 - val_accuracy: 0.9102
Epoch 10/10
1688/1688 [==============================] - 217s 129ms/step - loss: 0.0221 - accuracy: 0.9924 - val